In [2]:
# coding: utf-8
"""
作ったライブラリの評価
"""
from filer2.filer2 import Filer
from library.submodular import *
from library.graphsubmodular import *
from library.vector import *
from library.rouge import *
import numpy as np
import collections
from scipy.sparse import csr_matrix, lil_matrix

### 英語版(Opinosis)

In [19]:
# ファイルの読み込み
dict_test = Filer.readdump('./files/OpinosisDataset1.0/preprocessed1/test/accuracy_garmin_nuvi_255W_gps.dump')
dict_ans = Filer.readdump('./files/OpinosisDataset1.0/preprocessed1/test/accuracy_garmin_nuvi_255W_gps.dump')

In [20]:
# テスト用ファイルの作成
# １００単語以上ある文は除く
list_sentence = []
list_sep = []
list_sep_all = []
list_edgelist = []
for sen, sep, sepall, edge in zip(dict_test['sentence'],
                                  dict_test['sep_njv'],
                                  dict_test['sep_all'],
                                  dict_test['edge_njv']):
    if len(sepall) <= 100 and len(sep) != 0:
        list_sentence.append(sen)
        list_sep.append(sep)
        list_sep_all.append(sepall)
        list_edgelist.append(edge)
        
list_edgelist = [[row1[0], row1[1]] for row in list_edgelist for row1 in row]

In [4]:
# インスタンス化
GM = GraphSubModular(list_sep_all=list_sep_all,
                     list_sep=list_sep,
                     list_edgelist=list_edgelist,
                     directed=True,
                     inverse_flag=True)
V = Vector(list_sep_all=list_sep_all,
           list_sep=list_sep,
           dict_path = './files/OpinosisDataset1.0/w2v/model_lemmas/model_cbow_s300_w5_m0/dict_word_vec_3')

SM = SubModular(list_sep_all=list_sep_all,
                list_sep=list_sep)

MGSM = Modified_GraphSubModular(list_sep_all=list_sep_all,
                                list_sep=list_sep,
                                list_edgelist=list_edgelist,
                                directed=False,
                                simrank_flag=True,
                                weighted=False,
                                log_flag=True)

num word:  869
num vocabulary:  336
Non dictionalized word: 

library/graphsubmodular.py:160: RuntimeWarning: Couldn't reach some vertices at structural_properties.c:4482
  output='epath')
library/graphsubmodular.py:637: RuntimeWarning: invalid value encountered in divide
  G = G / np.sum(G, axis=1, dtype=float)[:,np.newaxis]


 0
num_sentence:  89
num word:  869
num vocabulary:  336
num word:  869
num vocabulary:  336
Non dictionalized word:  0


In [22]:
MV = Modified_Vector(list_sep_all=list_sep_all,
                     list_sep=list_sep,
                     dict_path='./files/OpinosisDataset1.0/w2v/model_tokens/model_cbow_s300_w5_m0/dict_word_vec_3')

[ 0.68802561  0.97329804]
[ 0.41414303  0.58585697]
[ 0.12509557  0.47315387  0.37328003  0.47315387  0.47315387  0.41014049
  0.32684134  0.26382796  0.47315387  0.47315387  0.47315387]
[ 0.02883643  0.10906916  0.08604672  0.10906916  0.10906916  0.09454362
  0.0753419   0.06081636  0.10906916  0.10906916  0.10906916]
[ 0.0860032   0.32529329  0.32529329  0.32529329  0.32529329  0.28197159
  0.32529329  0.32529329  0.32529329  0.32529329  0.32529329  0.32529329
  0.12166226  0.32529329  0.32529329  0.32529329]
[ 0.018227    0.06894071  0.06894071  0.06894071  0.06894071  0.05975937
  0.06894071  0.06894071  0.06894071  0.06894071  0.06894071  0.06894071
  0.02578437  0.06894071  0.06894071  0.06894071]
[ 0.47315387  0.47315387  0.47315387  0.41014049  0.37328003  0.37328003
  0.27340619  0.47315387  0.47315387  0.47315387  0.31026665]
[ 0.10332457  0.10332457  0.10332457  0.08956408  0.08151471  0.08151471
  0.05970484  0.10332457  0.10332457  0.10332457  0.06775422]
[ 0.32225325  0.

In [5]:
# 学習
GM.m_greedy(num_w=100, r=1, scale=0)
V.m_greedy(num_w=100, r=1, scale=0)

計算が終了しました
計算が終了しました


In [6]:
SM.m_greedy(num_w=100, alpha=0.1, lamda=6, n_cluster=10)

計算が終了しました


In [7]:
MGSM.m_greedy(num_w=100, r=1, scale=0)
MV.m_greedy(num_w=100, r=1, scale=0)

計算が終了しました
計算が終了しました


In [8]:
# 文書数のチェック
print 'GM:'
print 'sentence: ', len(GM.list_C)
for row in GM.list_C:
    print len(row[2])
print 'V:'
print 'sentence: ', len(V.list_C)
for row in V.list_C:
    print len(row[2])
print 'SM:'
print 'sentence: ', len(SM.list_C)
for row in SM.list_C:
    print len(row[2])
print 'MGSM:'
print 'sentence: ', len(MGSM.list_C)
for row in MGSM.list_C:
    print len(row[2])
print 'MV:'
print 'sentence: ', len(MV.list_C)
for row in MV.list_C:
    print len(row[2])

GM:
sentence:  property: list_C
5
property: list_C
52
7
9
22
8
V:
sentence:  property: list_C
9
property: list_C
30
5
7
10
15
9
7
11
5
SM:
sentence:  property: list_C
12
property: list_C
6
6
5
5
8
5
10
11
9
12
9
9
MGSM:
sentence:  property: list_C
9
property: list_C
5
6
10
9
12
7
11
22
16
MV:
sentence:  property: list_C
8
property: list_C
52
5
5
5
6
11
9
7


In [9]:
for row in GM.list_C:
    print row[0]
for row in V.list_C:
    print row[0]
for row in SM.list_C:
    print row[0]
for row in MGSM.list_C:
    print row[0]
for row in MV.list_C:
    print row[0]

property: list_C
11
66
88
18
85
property: list_C
87
86
66
57
30
45
36
24
21
property: list_C
76
34
86
21
85
15
57
61
45
41
55
88
property: list_C
86
34
57
45
26
36
29
18
56
property: list_C
5
86
15
21
34
29
45
36


In [36]:
# 単語数のチェック
counter = 0
for row in SM.list_C:
    counter += len(row[2])
print counter

property: list_C
100


In [13]:
# Rougeを計算するようにbag_of_wordsを作成する
list_gm_test = [row[2] for row in GM.list_C]
list_v_test = [row[2] for row in V.list_C]
list_sm_test = [row[2] for row in SM.list_C]
list_mgm_test = [row[2] for row in MGM.list_C]
list_mv_test = [row[2] for row in GM.list_C]

property: list_C
property: list_C
property: list_C
property: list_C
property: list_C


In [19]:
num=4
print Rouge_N.rouge(list_gm_test, dict_ans['sep_all'][num], N=1)
print Rouge_N.rouge(list_v_test, dict_ans['sep_all'][num], N=1)
print Rouge_N.rouge(list_sm_test, dict_ans['sep_all'][num], N=1)
print Rouge_N.rouge(list_mgm_test, dict_ans['sep_all'][num], N=1)
print Rouge_N.rouge(list_mv_test, dict_ans['sep_all'][num], N=1)

0.68
0.64
0.6
0.68
0.68


In [41]:
b = np.array([1,2,3,4])
#a = np.array([2,3,4,5])
a = np.c_[a,b]
print a

[[2 1 1 1 1 1 1 1]
 [3 2 2 2 2 2 2 2]
 [4 3 3 3 3 3 3 3]
 [5 4 4 4 4 4 4 4]]


In [7]:
a

array([], dtype=float64)